<a href="https://colab.research.google.com/github/varunvenkatsaiis/colab_notebooks/blob/main/using_word_embeddings_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

--2025-03-21 06:00:21--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  16.6MB/s    in 8.2s    

2025-03-21 06:00:30 (9.79 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [ ]:
!tar -xf aclImdb_v1.tar.gz

In [ ]:
aclImdb/

SyntaxError: invalid syntax (<ipython-input-3-9219b10e8f06>, line 1)

In [ ]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
cd aclImdb/

/content/aclImdb


In [ ]:
ls


imdbEr.txt  imdb.vocab  README  test/  train/


In [ ]:
cd ..

/content


In [ ]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
cd aclImdb/

/content/aclImdb


In [ ]:
cd test/

/content/aclImdb/test


In [ ]:
ls


labeledBow.feat  neg/  pos/  urls_neg.txt  urls_pos.txt


In [ ]:
cd ../..

/content


In [ ]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
cd aclImdb/

/content/aclImdb


In [ ]:
cd train/

/content/aclImdb/train


In [ ]:
ls


labeledBow.feat  neg/  pos/  unsup/  unsupBow.feat  urls_neg.txt  urls_pos.txt  urls_unsup.txt


In [ ]:
cd ../../..

/


In [ ]:
ls

bin@                        dev/     lib32@   NGC-DL-CONTAINER-LICENSE  root/  tmp/
boot/                       etc/     lib64@   opt/                      run/   tools/
content/                    home/    libx32@  proc/                     sbin@  usr/
cuda-keyring_1.1-1_all.deb  kaggle/  media/   python-apt/               srv/   var/
datalab/                    lib@     mnt/     python-apt.tar.xz*        sys/


In [ ]:
cd content/

/content


In [ ]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
!rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os , pathlib , shutil , random


In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("pos" , "neg"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples : ]
  for fname in val_files:
    shutil.move( train_dir / category / fname ,
                val_dir / category / fname)



In [ ]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
cd aclImdb/

/content/aclImdb


In [ ]:
ls


imdbEr.txt  imdb.vocab  README  test/  train/  val/


In [ ]:
cd val/

/content/aclImdb/val


In [ ]:
ls

neg/  pos/


In [ ]:
cd ../../

/content


In [ ]:
ls


aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [ ]:
from tensorflow import keras

In [ ]:
batch_size = 32

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/train" , batch_size = batch_size
)

Found 20000 files belonging to 2 classes.


In [ ]:
val_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/val" , batch_size = batch_size
)

Found 5000 files belonging to 2 classes.


In [ ]:
test_ds = keras.utils.text_dataset_from_directory(

    "aclImdb/test" , batch_size = batch_size
)

Found 25000 files belonging to 2 classes.


In [ ]:
print(val_ds)

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
val_ds.class_names

['neg', 'pos']

In [ ]:
val_ds.shape

AttributeError: '_PrefetchDataset' object has no attribute 'shape'

In [ ]:
for i in val_ds:
  print(i)
  break;

(<tf.Tensor: shape=(32,), dtype=string, numpy=
array([b'I have seen and liked the original film, and expected more from a remake than this.<br /><br />GOOD: Effects and makeup are good. No complaints about the score and visuals, they are adequate, and the performances were okay (Tim Roth was excellent, the other principals were fine, and a handful of the "supporting supporting actors" did very well with extremely limited roles). The action scenes were exciting and fun.<br /><br />BAD: The escape from the ape city was terrible. The characters are going in circles, then suddenly someplace in the middle of town there are tunnels to escape. Plus, what escape route leads through everybody\'s bedroom?<br /><br />The story was pared down to include as much action as possible. I like action scenes but the original film had more meat to it and deserved more respect.<br /><br />Finally, the ending was completely nonsensical as presented. Without seeing the inevitable sequel, there is no justific

In [ ]:
from tensorflow.keras.layers import  TextVectorization

In [ ]:
text_vectorization =  TextVectorization(
    max_tokens = 20000,
    output_mode = "multi_hot"

)

In [ ]:
text_only_train_ds = train_ds.map(lambda x , y : x)

In [ ]:
 text_vectorization.adapt(text_only_train_ds)

In [ ]:
bin_1gram_train_ds = train_ds.map(
    lambda x , y : ( text_vectorization(x) , y) , num_parallel_calls = 4
)

In [ ]:
print(bin_1gram_train_ds)

<_ParallelMapDataset element_spec=(TensorSpec(shape=(None, 20000), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


32
